# Transactions from multiple systems

## 1. Setup

In [4]:
import os
import math
import json
import pytz

from datetime import datetime, timedelta
from typing import List, Tuple, Dict
from collections import namedtuple

import lusid
import pandas as pd

from lusid import models as models
from lusidjam import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from IPython.core.display import HTML

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = "{:,.2f}".format
display(HTML("<style>.container { width:90% !important; }</style>"))

secrets_path = '/Users/msingh/Projects/lusidws/secrets.json'

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

In [5]:
portfolios_api = api_factory.build(lusid.api.PortfoliosApi)
transactions_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
properties_api = api_factory.build(lusid.api.PropertyDefinitionsApi)

## 2. Create Portfolios

In [6]:
from lusid.exceptions import ApiException

def check_portfolio(scope, code):
    try:
        portfolios_api.delete_portfolio(scope=scope, code=code)
        return 'Portfolio successfully deleted'
    except ApiException as e:
        return 'Portfolio does not yet exist'

In [7]:
from datetime import date
import lusid.models as lm

def create_portfolio(portfolio_code, scope='', display_name='', base_currency='GBP'):
    check_portfolio(scope=scope, code=portfolio_code)
    request = lm.CreateTransactionPortfolioRequest(
        display_name=display_name,
        code=portfolio_code,
        base_currency=base_currency,
        created=str(date(2021, 10, 10)))
    
    try:
        result = transactions_api.create_portfolio(
            scope=scope,
            create_transaction_portfolio_request=request)
        return result
    except ApiException as e:
        display(json.loads(e.body)['title'])

In [9]:
port_1250_response = create_portfolio('1250', scope='Bleugrain', display_name='Portfolio 1250', base_currency='GBP')
port_1232_response = create_portfolio('1232', scope='Bleugrain', display_name='Portfolio 1232', base_currency='GBP')
port_6350_response = create_portfolio('6350', scope='Bleugrain', display_name='Portfolio 6350', base_currency='GBP')

## 3. Create instruments

In [10]:
instruments_api = api_factory.build(lusid.api.InstrumentsApi)

In [11]:
InstrumentSpec = namedtuple('InstrumentSpec', ['Figi', 'Name'])

instruments = [
    InstrumentSpec("BBG000FD8G46", "HISCOX LTD"),
    InstrumentSpec("BBG000DW76R4", "ITV PLC"),
    InstrumentSpec("BBG000PQKVN8", "MONDI PLC"),
    InstrumentSpec("BBG000BDWPY0", "NEXT PLC"),
    InstrumentSpec("BBG000BF46Y8", "TESCO PLC"),
]

instruments_to_create = {
    i.Figi: models.InstrumentDefinition(
        name=i.Name, identifiers={ 'Figi': models.InstrumentIdValue(value=i.Figi) }
    ) for i in instruments
}

response = instruments_api.upsert_instruments(request_body=instruments_to_create)

instrument_ids = sorted([i.lusid_instrument_id for i in response.values.values()])
instrument_ids

['LUID_00003D6R',
 'LUID_00003D6U',
 'LUID_00003D6V',
 'LUID_00003D6W',
 'LUID_00003D6X']

In [12]:
import random

transactions = [
    models.TransactionRequest(
        transaction_id=f'TransactionId_{_id}',
        type='Buy',
        instrument_identifiers={ 'Instrument/default/LusidInstrumentId': _id },
        transaction_date=datetime.now(pytz.UTC).isoformat(),
        settlement_date=(datetime.now(pytz.UTC) + timedelta(days=2)).isoformat(),
        units=math.floor(random.random() * 100),
        total_consideration=lusid.CurrencyAndAmount(
            math.floor(random.random() * 1000), 'GBP')
    )
    for _id in instrument_ids
]

In [22]:
scope = 'Bleugrain'
portfolio_code = '1250'

response = transactions_api.upsert_transactions(
    scope, portfolio_code, transactions
)

## 4. Add property to the transactions

In [23]:
property_domain = 'Transaction'

common_scope = 'common'
ps_scope = 'ps'
crims_scope = 'crims'

default_properties = ['A', 'B']
ps_properties = ['C', 'D']
crims_properties = ['E', 'F']

In [24]:
def create_property(code, domain='Transaction', scope='default', required=False):
    try:
        properties_api.create_property_definition(
            create_property_definition_request=models.CreatePropertyDefinitionRequest(
                domain=domain,
                scope=scope,
                code=code,
                display_name=code,
                life_time='Perpetual',
                value_required=required,
                data_type_id=models.resource_id.ResourceId(
                    scope='system',
                    code='string',
                )
            )
        )
    
    except ApiException as e:
        detail = json.loads(e.body)
        if detail['code'] != 124:
            raise e

In [25]:
# create default properties common to PS and CRIMS
create_property(default_properties[0], scope=common_scope, required=True)
create_property(default_properties[1], scope=common_scope, required=True)

create_property(ps_properties[0], scope=ps_scope)
create_property(ps_properties[1], scope=ps_scope)

create_property(crims_properties[0], scope=crims_scope)
create_property(crims_properties[1], scope=crims_scope)

In [26]:
def update_property(transaction_id, property_code, property_value, portfolio_code='1250', 
                    scope='Bleugrain', property_domain='Transaction', property_scope='common'):
    print(f'{property_domain}/{property_scope}/{property_code}')
    perpetual_property = models.PerpetualProperty(
        f'{property_domain}/{property_scope}/{property_code}', 
        models.PropertyValue(label_value=property_value)
    )
    
    transactions_api.upsert_transaction_properties(
        scope=scope,
        code=portfolio_code,
        transaction_id=transaction_id,
        request_body={ f'{property_domain}/{property_scope}/{property_code}': perpetual_property }
    )

In [27]:
transaction_ids = [t.transaction_id for t in transactions]
transaction_ids

['TransactionId_LUID_00003D6R',
 'TransactionId_LUID_00003D6U',
 'TransactionId_LUID_00003D6V',
 'TransactionId_LUID_00003D6W',
 'TransactionId_LUID_00003D6X']

### 4.1 1st transaction

In [28]:
# 1st transaction
# default properties
update_property(transaction_ids[0], default_properties[0], f'Value_{transaction_ids[0]}_{default_properties[0]}')
update_property(transaction_ids[0], default_properties[1], f'Value_{transaction_ids[0]}_{default_properties[1]}')

# PS properties
update_property(transaction_ids[0], ps_properties[0], f'Value_{transaction_ids[0]}_{ps_properties[0]}', property_scope=ps_scope)

# CRIMS properties
update_property(transaction_ids[0], crims_properties[1], f'Value_{transaction_ids[0]}_{crims_properties[1]}', property_scope=crims_scope)

Transaction/common/A
Transaction/common/B
Transaction/ps/C
Transaction/crims/F


In [29]:
# 1st transaction update
# default properties

update_property(transaction_ids[0], default_properties[0], f'Value_{transaction_ids[0]}_{default_properties[0]}_updated')
# update_property(transaction_ids[0], default_properties[1], f'Value_{transaction_ids[0]}_{default_properties[1]}')

# PS properties
update_property(transaction_ids[0], ps_properties[0], f'Value_{transaction_ids[0]}_{ps_properties[0]}_updated', property_scope=ps_scope)

# CRIMS properties
# update_property(transaction_ids[0], crims_properties[1], f'Value_{transaction_ids[0]}_{crims_properties[1]}', property_scope=crims_scope)

Transaction/common/A
Transaction/ps/C


### 4.2 2nd transaction

In [30]:
# 2nd transaction
# default properties
update_property(transaction_ids[1], default_properties[0], f'Value_{transaction_ids[1]}_{default_properties[0]}')
update_property(transaction_ids[1], default_properties[1], f'Value_{transaction_ids[1]}_{default_properties[1]}')

# PS properties
update_property(transaction_ids[1], ps_properties[1], f'Value_{transaction_ids[1]}_{ps_properties[1]}', property_scope=ps_scope)

# CRIMS properties
update_property(transaction_ids[1], crims_properties[0], f'Value_{transaction_ids[1]}_{crims_properties[0]}', property_scope=crims_scope)

Transaction/common/A
Transaction/common/B
Transaction/ps/D
Transaction/crims/E


In [31]:
# 2nd transaction update
# default properties
# update_property(transaction_ids[1], default_properties[0], f'Value_{transaction_ids[1]}_{default_properties[0]}')
# update_property(transaction_ids[1], default_properties[1], f'Value_{transaction_ids[1]}_{default_properties[1]}')

# PS properties
# update_property(transaction_ids[1], ps_properties[1], f'Value_{transaction_ids[1]}_{ps_properties[1]}', property_scope=ps_scope)

# CRIMS properties
update_property(transaction_ids[1], crims_properties[0], f'Value_{transaction_ids[1]}_{crims_properties[0]}_updated', property_scope=crims_scope)

Transaction/crims/E


### 4.3 3rd transaction

In [32]:
# 3rd transaction
# default properties
update_property(transaction_ids[2], default_properties[0], f'Value_{transaction_ids[2]}_{default_properties[0]}')
update_property(transaction_ids[2], default_properties[1], f'Value_{transaction_ids[2]}_{default_properties[1]}')

# PS properties
update_property(transaction_ids[2], ps_properties[0], f'Value_{transaction_ids[2]}_{ps_properties[0]}', property_scope=ps_scope)
update_property(transaction_ids[2], ps_properties[1], f'Value_{transaction_ids[2]}_{ps_properties[1]}', property_scope=ps_scope)

# CRIMS properties
update_property(transaction_ids[2], crims_properties[0], f'Value_{transaction_ids[2]}_{crims_properties[0]}', property_scope=crims_scope)
update_property(transaction_ids[2], crims_properties[1], f'Value_{transaction_ids[2]}_{crims_properties[1]}', property_scope=crims_scope)

Transaction/common/A
Transaction/common/B
Transaction/ps/C
Transaction/ps/D
Transaction/crims/E
Transaction/crims/F


In [33]:
# 3rd transaction update
# default properties
# update_property(transaction_ids[2], default_properties[0], f'Value_{transaction_ids[2]}_{default_properties[0]}')
# update_property(transaction_ids[2], default_properties[1], f'Value_{transaction_ids[2]}_{default_properties[1]}')

# PS properties
update_property(transaction_ids[2], ps_properties[0], f'Value_{transaction_ids[2]}_{ps_properties[0]}_updated', property_scope=ps_scope)
update_property(transaction_ids[2], ps_properties[1], f'Value_{transaction_ids[2]}_{ps_properties[1]}_updated', property_scope=ps_scope)

# CRIMS properties
# update_property(transaction_ids[2], crims_properties[0], f'Value_{transaction_ids[2]}_{crims_properties[0]}', property_scope=crims_scope)
# update_property(transaction_ids[2], crims_properties[1], f'Value_{transaction_ids[2]}_{crims_properties[1]}', property_scope=crims_scope)

Transaction/ps/C
Transaction/ps/D


## 5. Retrieve transactions

In [34]:
lusid_response_to_data_frame(
    transactions_api.get_transactions(scope=scope, 
                                      code=portfolio_code, 
                                      property_keys=[ 
                                          'Transaction/common/A', 
                                          'Transaction/common/B', 
                                          'Transaction/ps/C', 
                                          'Transaction/ps/D', 
                                          'Transaction/crims/E', 
                                          'Transaction/crims/F']),
    rename_properties=True,
)

,transaction_id,type,instrument_identifiers.Instrument/default/LusidInstrumentId,instrument_scope,instrument_uid,transaction_date,settlement_date,units,transaction_price.price,transaction_price.type,total_consideration.amount,total_consideration.currency,exchange_rate,transaction_currency,A(common-Properties),B(common-Properties),C(ps-Properties),F(crims-Properties),entry_date_time,D(ps-Properties),E(crims-Properties),properties
0,TransactionId_LUID_00003D6R,Buy,LUID_00003D6R,default,LUID_00003D6R,2021-12-29 10:42:19.670391+00:00,2021-12-31 10:42:19.670407+00:00,91.00,0.00,Price,943.00,GBP,1.00,GBP,Value_TransactionId_LUID_00003D6R_A_updated,Value_TransactionId_LUID_00003D6R_B,Value_TransactionId_LUID_00003D6R_C_updated,Value_TransactionId_LUID_00003D6R_F,2021-12-29 10:47:09.990856+00:00,NaN,NaN,NaN
1,TransactionId_LUID_00003D6U,Buy,LUID_00003D6U,default,LUID_00003D6U,2021-12-29 10:42:19.670566+00:00,2021-12-31 10:42:19.670574+00:00,92.00,0.00,Price,993.00,GBP,1.00,GBP,Value_TransactionId_LUID_00003D6U_A,Value_TransactionId_LUID_00003D6U_B,NaN,NaN,2021-12-29 10:47:15.979791+00:00,Value_TransactionId_LUID_00003D6U_D,Value_TransactionId_LUID_00003D6U_E_updated,NaN
2,TransactionId_LUID_00003D6V,Buy,LUID_00003D6V,default,LUID_00003D6V,2021-12-29 10:42:19.670667+00:00,2021-12-31 10:42:19.670673+00:00,91.00,0.00,Price,537.00,GBP,1.00,GBP,Value_TransactionId_LUID_00003D6V_A,Value_TransactionId_LUID_00003D6V_B,Value_TransactionId_LUID_00003D6V_C_updated,Value_TransactionId_LUID_00003D6V_F,2021-12-29 10:47:24.039873+00:00,Value_TransactionId_LUID_00003D6V_D_updated,Value_TransactionId_LUID_00003D6V_E,NaN
3,TransactionId_LUID_00003D6W,Buy,LUID_00003D6W,default,LUID_00003D6W,2021-12-29 10:42:19.670758+00:00,2021-12-31 10:42:19.670764+00:00,17.00,0.00,Price,867.00,GBP,1.00,GBP,NaN,NaN,NaN,NaN,2021-12-29 10:46:26.791568+00:00,NaN,NaN,{}
4,TransactionId_LUID_00003D6X,Buy,LUID_00003D6X,default,LUID_00003D6X,2021-12-29 10:42:19.670847+00:00,2021-12-31 10:42:19.670853+00:00,12.00,0.00,Price,65.00,GBP,1.00,GBP,NaN,NaN,NaN,NaN,2021-12-29 10:46:26.791568+00:00,NaN,NaN,{}


In [35]:
lusid_response_to_data_frame(
    transactions_api.get_holdings(scope=scope, 
                                      code=portfolio_code),
    rename_properties=True,
)

,instrument_scope,instrument_uid,sub_holding_keys,SourcePortfolioId(default-Properties),SourcePortfolioScope(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency,currency,transaction.transaction_id,transaction.type,transaction.instrument_identifiers.Instrument/default/LusidInstrumentId,transaction.instrument_scope,transaction.instrument_uid,transaction.transaction_date,transaction.settlement_date,transaction.units,transaction.transaction_price.price,transaction.transaction_price.type,transaction.total_consideration.amount,transaction.total_consideration.currency,transaction.exchange_rate,transaction.transaction_currency,transaction.properties.Transaction/common/A.key,transaction.properties.Transaction/common/A.value.label_value,transaction.properties.Transaction/common/B.key,transaction.properties.Transaction/common/B.value.label_value,transaction.properties.Transaction/ps/C.key,transaction.properties.Transaction/ps/C.value.label_value,transaction.properties.Transaction/crims/F.key,transaction.properties.Transaction/crims/F.value.label_value,transaction.properties.Transaction/default/ResultantHolding.key,transaction.properties.Transaction/default/ResultantHolding.value.metric_value.value,transaction.entry_date_time,transaction.properties.Transaction/ps/D.key,transaction.properties.Transaction/ps/D.value.label_value,transaction.properties.Transaction/crims/E.key,transaction.properties.Transaction/crims/E.value.label_value
0,default,LUID_00003D6R,{},1250,Bleugrain,P,91.00,0.00,943.00,GBP,943.00,GBP,GBP,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
1,default,LUID_00003D6U,{},1250,Bleugrain,P,92.00,0.00,993.00,GBP,993.00,GBP,GBP,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2,default,LUID_00003D6V,{},1250,Bleugrain,P,91.00,0.00,537.00,GBP,537.00,GBP,GBP,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
3,default,LUID_00003D6W,{},1250,Bleugrain,P,17.00,0.00,867.00,GBP,867.00,GBP,GBP,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
4,default,LUID_00003D6X,{},1250,Bleugrain,P,12.00,0.00,65.00,GBP,65.00,GBP,GBP,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
5,default,CCY_GBP,{},1250,Bleugrain,C,-943.00,0.00,-943.00,GBP,-943.00,GBP,GBP,TransactionId_LUID_00003D6R,Buy,LUID_00003D6R,default,LUID_00003D6R,2021-12-29 10:42:19.670391+00:00,2021-12-31 10:42:19.670407+00:00,91.00,0.00,Price,943.00,GBP,1.00,GBP,Transaction/common/A,Value_TransactionId_LUID_00003D6R_A_updated,Transaction/common/B,Value_TransactionId_LUID_00003D6R_B,Transaction/ps/C,Value_TransactionId_LUID_00003D6R_C_updated,Transaction/crims/F,Value_TransactionId_LUID_00003D6R_F,Transaction/default/ResultantHolding,91.00,2021-12-29 10:47:09.990856+00:00,NaN,NaN,NaN,NaN
6,default,CCY_GBP,{},1250,Bleugrain,C,-993.00,0.00,-993.00,GBP,-993.00,GBP,GBP,TransactionId_LUID_00003D6U,Buy,LUID_00003D6U,default,LUID_00003D6U,2021-12-29 10:42:19.670566+00:00,2021-12-31 10:42:19.670574+00:00,92.00,0.00,Price,993.00,GBP,1.00,GBP,Transaction/common/A,Value_TransactionId_LUID_00003D6U_A,Transaction/common/B,Value_TransactionId_LUID_00003D6U_B,NaN,NaN,NaN,NaN,Transaction/default/ResultantHolding,92.00,2021-12-29 10:47:15.979791+00:00,Transaction/ps/D,Value_TransactionId_LUID_00003D6U_D,Transaction/crims/E,Value_TransactionId_LUID_00003D6U_E_updated
7,default,CCY_GBP,{},1250,Bleugrain,C,-537.00,0.00,-537.00,GBP,-537.00,GBP,GBP,TransactionId_LUID_00003D6V,Buy,LUID_00003D6V,default,LUID_00003D6V,2021-12-29 10:42:19.670667+00:00,2021-12-31 10:42:19.670673+00:00,91.00,0.00,Price,537.00,GBP,1.00,GBP,Transaction/common/A,Value_TransactionId_LUID_00003D6V_A,Transaction/common/B,Value_Tran

In [20]:
create_property(crims_properties[1], domain='Holding', scope=crims_scope)

In [45]:
transactions_api.
# update_property(transaction_ids[0], default_properties[0], f'Holding_Value_{transaction_ids[0]}_{default_properties[0]}', property_domain='Holding')

Holding/common/A


ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Wed, 29 Dec 2021 08:19:18 GMT', 'Content-Type': 'application/problem+json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'X-Rate-Limit-Limit': '1m', 'X-Rate-Limit-Remaining': '4999', 'X-Rate-Limit-Reset': '2021-12-29T08:20:17.8245781Z', 'lusid-meta-success': 'False', 'lusid-meta-requestId': '0HME7483L5QNT:00000007', 'lusid-meta-correlationId': '0HME7483L5QNT:00000007', 'lusid-meta-duration': '276', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains', 'Server': 'FINBOURNE', 'Content-Security-Policy': "default-src 'self' https://*.lusid.com https://*.finbourne.com; script-src 'unsafe-inline' 'self' https://*.lusid.com https://*.finbourne.com; font-src 'self' fonts.googleapis.com; img-src data: 'self' https://*.lusid.com https://*.finbourne.com; style-src 'unsafe-inline' 'self' https://*.lusid.com https://*.finbourne.com; report-uri https://lusid.report-uri.com/r/d/csp/enf